In [25]:
!pip install tqdm
!pip install tables
from tqdm.notebook import tqdm

     |████████████████████████████████| 71kB 3.4MB/s eta 0:00:011
     |████████████████████████████████| 4.3MB 4.2MB/s eta 0:00:01


Note: The cells that parse ngram files take an hour to run. 

In [26]:
import os
import csv
import pandas as pd

In [27]:
JSTOR_HOME = "../../../jstor_data"
INDICES = "./filtered_index.csv"

demographic_authors = ['hannan freeman', 'barnett carroll', 'barron west', 'brüderl schüssler', 'carrol hannan', 
                       'freeman carrol', 'fichman levinthal', 'carrol']
## Will leaving in single authors catch extra citations?

relational_authors = ['pfeffer salancik', 'burt christman', 'pfeffer nowak', 'pfeffer']

cultural_authors = ['meyer rowan', 'dimaggio powell', 'powell dimaggio', 'oliver', 'powell', 'scott', 'weick']

ALL_AUTHORS = set(demographic_authors + relational_authors + cultural_authors)

with open(INDICES, 'r') as f:
    files = f.read().split('\n')[:-1]


In [39]:
expanded_dict_folder = "../Dictionaries/Expanded/wordnet_english2/"
full_cultural = expanded_dict_folder + "closest_culture_1000.csv"
full_relational = expanded_dict_folder + "closest_relational_1000.csv"
full_demographic = expanded_dict_folder + "closest_demographic_1000.csv"
full_cultural_set = set()
full_relational_set = set()
full_demographic_set = set()


with open(full_cultural, 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        full_cultural_set.add(' '.join(line[0].split('_')))
with open(full_demographic, 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        full_demographic_set.add(' '.join(line[0].split('_')))
with open(full_relational, 'r') as f:
    reader = csv.reader(f)
    for line in reader:
        full_relational_set.add(' '.join(line[0].split('_')))
        
all_terms = set.union(full_cultural_set, full_relational_set, full_demographic_set)

In [40]:
counts_df = pd.DataFrame(columns=["article_id", "cultural_author_count", "demographic_author_count", "relational_author_count",
                                 "cultural_count2", "relational_count2", "demographic_count2",
                                 "cultural_count1", "relational_count1", "demographic_count1"]) 
NGRAM = 2 
folder = os.path.join(JSTOR_HOME, 'ngram{}'.format(NGRAM))

for file in tqdm(files):
    with open(os.path.join(folder, '{}-ngram{}.txt'.format(file, NGRAM)), 'r') as f:
        d = {}
        row = {"article_id": file}

        for line in f.read().splitlines():
            k, v = line.split('\t')
            if k in ALL_AUTHORS or k in all_terms:
                d[k] = int(v)

        row["cultural_author_count"] = sum([d.get(author, 0) for author in cultural_authors])
        row["demographic_author_count"] = sum([d.get(author, 0) for author in demographic_authors])
        row["relational_author_count"] = sum([d.get(author, 0) for author in relational_authors])
        row["cultural_count2"] = sum([d.get(term, 0) for term in full_cultural_set])
        row["demographic_count2"] = sum([d.get(term, 0) for term in full_demographic_set])
        row["relational_count2"] = sum([d.get(term, 0) for term in full_relational_set])

        counts_df = counts_df.append(row, ignore_index=True)


        

In [43]:
counts_df = counts_df.set_index('article_id')
counts_df.head()

,cultural_author_count,demographic_author_count,relational_author_count,cultural_count2,relational_count2,demographic_count2,cultural_count1,relational_count1,demographic_count1
article_id,,,,,,,,,
journal-article-10.2307_2065002,0,0,0,0,2,1,NaN,NaN,NaN
journal-article-10.2307_3380821,0,0,0,0,1,0,NaN,NaN,NaN
journal-article-10.2307_2095822,0,0,0,0,0,0,NaN,NaN,NaN
journal-article-10.2307_2631839,0,0,0,0,0,0,NaN,NaN,NaN
journal-article-10.2307_40836133,0,0,0,0,40,33,NaN,NaN,NaN


In [44]:
NGRAM = 1 
folder = os.path.join(JSTOR_HOME, 'ngram{}'.format(NGRAM))

for file in tqdm(files):
    with open(os.path.join(folder, '{}-ngram{}.txt'.format(file, NGRAM)), 'r') as f:
        d = {}
        for line in f.read().splitlines():
            k, v = line.split('\t')
            if k in ALL_AUTHORS or k in all_terms:
                d[k] = int(v)
        counts_df.at[file, "cultural_author_count"] = sum([d.get(author, 0) for author in cultural_authors])
        counts_df.at[file, "demographic_author_count"] = sum([d.get(author, 0) for author in demographic_authors])
        counts_df.at[file, "relational_author_count"] = sum([d.get(author, 0) for author in relational_authors])
        
        counts_df.at[file, "cultural_count1"] = sum([d.get(term, 0) for term in full_cultural_set])
        counts_df.at[file, "demographic_count1"] = sum([d.get(term, 0) for term in full_demographic_set])
        counts_df.at[file, "relational_count1"] = sum([d.get(term, 0) for term in full_relational_set])


In [45]:
counts_df.sum()

cultural_author_count          60543.0
demographic_author_count         264.0
relational_author_count        10467.0
cultural_count2                52010.0
relational_count2             410184.0
demographic_count2            278936.0
cultural_count1              7994280.0
relational_count1           10019502.0
demographic_count1           5484991.0
dtype: float64

In [47]:
counts_df.to_csv('citation_and_expanded_dict_count_may7.csv', index=True)